# Paquete DataFrames


## DataFrames.jl
Es un paquete que nos permite explorar información tabulándola. Este paquete es alternativo y equivalente al famoso paquete 'Pandas' de Python.

Como el nombre del paquete lo indica, la información se maneja en una estructuta de datos llamada 'dataframe', el cual es similar a una tabla u hoja de cálculo (como la de Excel). Por lo tanto, el paquete nos permite explorar un dataset, transformarlo y almacenar la información.

Junto a [`DataFrames`](https://dataframes.juliadata.org/v0.17/##DataFrames.jl-1) se puede utilizar el paquete [`Queryverse`](https://www.queryverse.org/), el cual permite tener a mano diferentes herramientas útiles para: trabajar con archivos, manipular datos y explorarlos; por tanto, permite tener mayor eficiencia.



In [388]:
using Pkg; Pkg.activate(".") ##Activando enviroment local



Los paquetes `DataFrames` y `Queryverse` pueden ser añadidos a Julia y luego ser cargados de la forma usual. Se utilizará el paquete Plots, para realizar algunas gráficas y de esta forma visualizar datos.



In [389]:
using Plots, Queryverse, DataFrames ##Importando librerías



### Cargando archivos
En Julia existen diversos paquetes que permiten cargar archivos, como el paquete `CSV`. Sin embargo, el paquete `Queryverse` permite usar la función `load`, que permite leer no solo archivos $^*$.csv sino también archivos: $^*$.xls, $^*$.xlsx, $^*$.dta, $^*$.sav, entre otros.

Ahora cargaremos el dataset que se estará utilizando para mostrar las bondades del paquete `DataFrames`.



In [390]:
datatrain = load("./titanic/train.csv");



A continuación se muestra cómo se carga los datos de una archivo $^*$.xlsx, en el cual se especifica el nombre de la hoja de cálculo (en el caso particular del ejemplo 'train').



In [391]:
datatest = load("./titanic/test.xlsx", "test");



Ahora podemos comprobar que los datos efectivamente ha sido cargado en las variables de ambos archivos.



In [392]:
typeof(datatrain)


In [393]:
datatest;



A menudo se requerirá que los datos deban ser contenidos en un dataframe, para ello se utiliza el constructor `DataFrame` del paquete `DataFrames`. Podrá notar la mejora en la presentación de la tablas cuando se utiliza el paquete mencionado. 

Existen varias maneras de crear el dataframe, la primera es:



In [394]:
dftest = DataFrame(datatest)



La segunda es más elegante y se utiliza pipe sintax:



In [395]:
dftrain = datatrain |> DataFrame;


In [396]:
typeof(datatrain), typeof(dftrain)



Como nota adicional, el método `load` también admite pasar como parámetro un `String` con la una url de donde el conjunto de datos pueda ser descargado, el archivo se guarda en el directorio que contiene el archivo que es ejecutado..




### Trabajando con los datos

A continuación, se discutirán sobre alguna de las funciones implementadas en DataFrames para trabajar con los datos tabulados. 




#### Obteniendo información del dataset
Existe un conjunto de funciones que nos permite tener un overview del dataset que se ha cargado. Lo anterior será importante para dar un manejo adecuado a los datos.

Para obtener las columnas del dataframe se utiliza el método `names`.



In [397]:
names(dftrain)


In [398]:
names(dftest)



Las dimensiones de dataframe se puede obtener con el método `size`.



In [399]:
size(dftrain)


In [400]:
size(dftest)



Una descripción del contenido de las columnas puede realizarse con el método `describe`. El método entrega información estadística de las columnas y varias propiedades.



In [401]:
describe(dftrain);


In [402]:
describe(dftest)



#### Renombrando las columnas

Renombrar una columna es útil cuando los nombres no son descriptivos y legibles.



In [403]:
rename!(dftrain, :PassengerId=>"Id", :Pclass=>"Class", :Sex=>"Gender")


In [404]:
rename!(dftest, :PassengerId=>"Id", :Pclass=>"Class", :Sex=>"Gender")



#### Accediendo a los elementos de un dataframe
Diferentes formas puede ser aplicado para acceder a los elementos de un dataframe. A continuación se muestran algunas.

**Mediante el punto (.col)**



In [405]:
dftrain.Name ###Retorna un array



**Mediante el símbolo ([:col])**



In [406]:
dftrain[:Age] ###Retorna un Array



**Mediante índice**



In [407]:
dftrain[1,5]  ###Un elemento específico


En el ejemplo anterior se accedió a solo un dato de la columna, para acceder a toda la columna ser realiza con (:)


In [408]:
dftrain[:,5] ###Retorna un Array


In [409]:
dftrain[:,[5]] ###Retorna un DataFrame



Se puede acceder a un conjunto de columnas a la vez



In [410]:
dftrain[:,[1,4,5,3]] ###Retorna un DataFrame


In [411]:
dftrain[:,[:Id,:Name,:Gender,:Class]] ###Retorna un DataFrame



Es posible que, en lugar de indicar cada columna y dependiendo del nombre que posean las columnas, estas puedan ser indicadas por medio de una expresión regular indicado con `r\"--\"`. Lo anterior trata de hacer match con los nombres de las columnas y cuando se encuentre alguna coincidencia del nombre de la columna con la regla descrita por la expresión regular esta es devuelta.

En los ejemplos anteriores se accedió a todas las filas de las columnas, sin embargo, puede accederse a un grupo limitado como se muestra a continuación



In [412]:
dftrain[50:100,[:Id,:Name,:Gender,:Class]];


In [413]:
dftrain[[50,75,100],[:Id,:Name,:Gender,:Class]];



**Mediante una condición**



In [414]:
dftrain[dftrain.Class.==2,:];



Note como en el ejemplo anterior se utiliza broadcasting. La manera en que funciona lo anterior es de la siguiente forma: 

Primero se genera un vector con valores booleanos.



In [415]:
arraybool = dftrain.Class.==2;



Luego el arreglo se pasa en el lugar del primer índice cuado se desea acceder a los elementos del dataframe, el arreglo filtra los datos que serán mostrados (todas las filas que coincidan con el valor booleano true).



In [416]:
dftrain[arraybool,:];



Para mayor legibilidad en el código, lo mejor será usar un conjunto de funciones que nos permiten hacer filtrado de la información, por ejemplo usar el método `filter`. Usar dichos métodos nos permitirá crear condiciones más robustas, complejas y claras.

En el ejemplo que sigue se utiliza el paquete `Pipe`, el cual mejora la utilidad del uso de `pipe`, teniendo la posibilidad de usar funciones y pasar argumentos.



In [417]:
Pkg.add("Pipe"); using Pipe;


In [418]:
@pipe dftrain |> filter(x->x.Class==2,_)



De igual manera, el paquete `Queryverse` proporciona la macro `@filter` para realizar el filtrado, como se muestra a continuación.



In [419]:
dftrain |> @filter _.Class==2



Para la primera coincidencia se puede usar la función `findfirst`



In [420]:
@pipe dftrain |> findfirst(x->x==2,_.Class) |> dftrain[_,:]



**Funciones del paquete**



In [421]:
first(dftrain,6);


In [422]:
last(dftrain,6);



#### Funciones útiles
El paquete `DataFrames` dispone de varias funciones que pueden ser de utilidad cuando se manipula y se transforman datos. A continuación se mencionan algunos de ellos.



In [423]:
@pipe dftrain |> colwise(sum,_[[:Survived,:Class]])



La función `colwise` realiza operaciones por columnas cuando es necesario, en este caso el ejemplo no traduce a algo útil, pero ilustra el uso de la función.




Para columnas que no son numéricas puede usarse la función `countmap`, que cuenta el número de veces que aparece un determinado elemento en la serie (columna) de datos. Para el uso se instala y se importa el paquete `StatsBase`.



In [424]:
Pkg.add("StatsBase"); using StatsBase


In [425]:
@pipe dftrain |> countmap(_[:Gender])


In [426]:
@pipe dftrain |> countmap(_[:Embarked ])



Otra función interesante es `unique`, que devuelve un array con los valores no duplicados.



In [427]:
genders = dftrain[:Gender] |> unique


In [428]:
dftrain[:TotalFamily] = dftrain.SibSp + dftrain.Parch


In [429]:
dftest[:TotalFamily] = dftest.SibSp + dftest.Parch


In [430]:
dftrain[:new] = dftrain.SibSp + dftrain.Parch; 


In [431]:
dftrain



En el ejemplo anterior una columna, con el nombre `TotalFamily`, fue añadida al dataframe. Una columna adicional es agregado, `new`, para ilustrar cómo se elimina.



In [432]:
select!(dftrain,Not([:new]))



Para eliminar una fila del dataframe se puede usar la función `delete!` y para añadir una fila se puede usar la función `push!`. En `delete!` se indica el dataframe y el índice de la fila que debe ser eliminada.

Existen otro conjunto de funciones que son de utilizada cuando se requiere realizar una consulta compleja en la data, el paquete `Queryverse` por contener el módulo `Query` ofrece opciones interesantes que pueden ser consultados en los siguientes enlaces: [link₁](https://www.queryverse.org/2019/02/02/query-v0.11/), [lin₂](https://www.queryverse.org/Query.jl/stable/standalonequerycommands/##Standalone-query-operators-1), [lin₃](https://www.queryverse.org/Query.jl/stable/linqquerycommands/#LINQ-Style-Query-Commands-1). 

Más funciones puede ser encontrado en la siguiente [introducción](https://en.wikibooks.org/wiki/Introducing_Julia/DataFrames) al paquete `DataFrames`.




### Manejo de fechas
Es usual que al indagar en los datos y transformarlos se manejen fechas. En Julia ello se realiza con el paquete `Dates`, el cual nos permite dar formato a las fechas y realizar operaciones con ellas.

Para construir un objeto tipo fecha se pueden usar los constructores `Date` y `DateTime`, los cuales diferen en la precisión (día y milisegundos respectivamente). 



In [433]:
Pkg.add("Dates"); using Dates


In [434]:
Date(2021), Date(2021,5), Date(2021,6,4)


In [435]:
DateTime(2021), DateTime(2021,5), DateTime(2021,6,4)



En algunas ocasiones se encontrará necesario dar formato a una cadena de caracteres que representan una fecha, para ello en los constructores se especifica el formato de la cadena de texto, en el cual existen diferentes opciones.



In [436]:
Date("2021-3","yyyy-m"), Date("2021/08/9","yy/mm/d"), Date("20210809","yyyymmdd"), Date("21-3","yyyy-m")



Si existen muchos datos de fecha a formatear y tienen un formato similar, es más eficiente crear el formato primero y luego usarlo en el constructor, por ejemplo: `dateformat=DateFormat(\"dd/mm/yyyy\")`.

En los ejemplos se puede notar que exite un error para la última fecha, cuando ocurre esas situaciones se debe parsear la cadena de texto para ser convertido a una cadena tipo fecha.



In [437]:
parse(Dates.Date,"2021-3",Dates.DateFormat("yyyy-m"))



Para conocer más sobre el manejo de fechas revisar esta [documentación](https://docs.julialang.org/en/v1/stdlib/Dates/##Dates).
md



### Respondiendo interrogantes y visualización de datos
Para ilustrar cómo se utilizan algunas de las funciones más avanzadas, mencionadas hacia el final de la sección  'Funciones útiles', podemos dar respuesta a las siguientes preguntas:
+ ¿Cuántas personas iban en el titanic?
+ ¿Cuántos hombres y mujeres sobrevivieron?
+ ¿Cuál fué el top 10 de edad que más sobrevivieron?
+ ¿Cuál fue el top 10 de edad que no lograron sobrevivir?


El paquete `StatPlots` ofrece una macro `@df`, el cual se usa junto con `Plots` para graficar datos de dataframe.



In [438]:
Pkg.add("StatsPlots"); using StatsPlots



A continuación una rutina para responder la primera interrogante.



In [439]:
begin ###Preparamos los datos para unir los dataframes
	survivedtest = @pipe load("./titanic/gender_submission.csv") |> 
					DataFrame |> 
					rename!(_,:PassengerId=>"Id") |>
					Float64.(_)
	dftest₁ = @from i in dftest begin
			  @join j in survivedtest on i.Id equals j.Id
			  @select {i.Id, j.Survived, i.Class,i.Name, i.Gender, i.Age, i.SibSp, i.Parch, i.Ticket, i.Fare, i.Cabin, i.Embarked, i.TotalFamily}
			  @collect DataFrame
	end
end;


In [440]:
df = vcat(dftrain[:,:], dftest₁[:,:]); ###Se deben unir los dataframes


In [441]:
describe(df);



#### ¿Cuántas personas iban en el titanic?



In [442]:
length(df[:Id])



#### ¿Cuántos hombres y mujeres sobrevivieron?



In [443]:
genders


In [444]:
👨 = df |> @filter(_.Gender=="male" && _.Survived==1) |> collect |> length


In [445]:
👩 = df |> @filter(_.Gender=="female" && _.Survived==1) |> collect |> length



#### ¿Cuál fué el top 10 de edad que más sobrevivieron?



In [446]:
agesurviver = df[df.Survived.==1, [:Age]] |> 
				dropmissing |> 
				@groupby(_[:Age]) |>
				@map({Edadesₛ=key(_), Countₛ=length(_)}) |>
				DataFrame;


In [447]:
agesurviver


In [448]:
@pipe sort!(agesurviver,[:Countₛ],rev=true) |> first(_,10)


In [449]:
@pipe df[df.Survived.==1, [:Age]] |> 
		dropmissing |> 
		groupby(_,:Age)  |> 
		combine(_,:Age=>length=>:Countₛ) |> 
		sort(_,[:Countₛ],rev=true) |> 
		first(_,10)



#### ¿Cuál fue el top 10 de edad que no lograron sobrevivir?



In [450]:
notagesurviver = df[df.Survived.==0, [:Age]] |> 
	  			 dropmissing |> 
				 @groupby(_[:Age]) |>
				 @map({Edadesₙₛ=key(_), Countₙₛ=length(_)}) |>
				 DataFrame;


In [451]:
@pipe sort!(notagesurviver,[:Countₙₛ],rev=true) |> first(_,10)


In [452]:
@pipe df[df.Survived.==0, [:Age]] |> 
		dropmissing |> 
		groupby(_,:Age)  |> 
		combine(_,:Age=>length=>:Countₛ) |> 
		sort(_,[:Countₛ],rev=true) |> 
		first(_,10)



#### Gráfica con Plots



In [453]:
pyplot()


In [454]:
dfplotₛ = DataFrame();


In [455]:
dfplotₛ[:Gender] = @pipe df[df.Survived.==1, [:Gender]] |> 
			map(x->x=="male" ? 0 : 1, _[:Gender])


In [456]:
dfplotₙₛ = DataFrame();


In [457]:
dfplotₙₛ[:Gender] = @pipe df[df.Survived.==0, [:Gender]] |> 
			map(x->x=="male" ? 0 : 1, _[:Gender])


In [458]:
@df dfplotₛ plot(:Gender, seriestype=:histogram, legend=:none, bar_width=0.9, xticks=([0.15,1.15],["Varón", "Mujer"]), title="Sobrevivientes")


In [459]:
@df dfplotₙₛ plot(:Gender, seriestype=:histogram, legend=:none, bar_width=0.9, xticks=([0.15,1.15],["Varón", "Mujer"]), title="No Sobrevivientes")


In [460]:
f = DataFrame(a = 1:10, b = 10 * rand(10), c = 10 * rand(10));


In [461]:
@df f plot(:a, [:b :c])



### Paquete PlutoUI
Este paquete es que agrega syntactic sugar para alguna de las funcionalidades de Pluto. [`PlutoUI`](https://github.com/fonsp/PlutoUI.jl) puede ser implementado para colocar widgets en el cuaderno y hacer más interactivo el programa que se escriba.



In [462]:
Pkg.add("PlutoUI"); using PlutoUI;


In [463]:
@bind gender Select(genders) 


In [464]:
@bind number Slider(1:9) 


In [465]:
gender, number



### Utilidades de `Queryverse`



In [466]:
datatrain |> @vlplot(:point,x=:Survived,y=:Pclass,color="Sex:n")



Es posible realizar una gráfica como se muestra en el ejemplo, donde no se usa `Plots.jl`.



In [467]:
datatrain |> Voyager



`Voyager` es una utilidad que permite explorar de forma rápida los gráficos, ofreciendo la posibilidad de visualizarlos con gráficos.

